In [1]:
from preproc_functions_controls import *

In [2]:
root_path = '/Users/dg519/Documents/normative_paper_github/data'
remote_data_folders='/data_ic3online_cognition'
supervised_data_folders=['/data_healthy_v1','/data_healthy_v2']
folder_structure=['/summary_data','/trial_data','/speech']
output_clean_folder ='/data_healthy_cleaned'
list_of_tasks = ['IC3_NVtrailMaking','IC3_Orientation', 'IC3_PearCancellation', 'IC3_rs_digitSpan', 'IC3_rs_spatialSpan', 'IC3_rs_PAL', 'IC3_rs_SRT', 'IC3_rs_CRT', 'IC3_SemanticJudgment', 'IC3_i4i_IDED', 'IC3_i4i_motorControl','IC3_calculation', 'IC3_GestureRecognition', 'IC3_AuditorySustainedAttention','IC3_BBCrs_blocks', 'IC3_Comprehension','IC3_rs_oddOneOut', 'IC3_TaskRecall']
list_of_questionnaires = ['q_IC3_demographics', 'q_IC3_fatigue','q_IC3_GDS','q_IC3_apathy']
list_of_speech = ['IC3_NamingTest','IC3_Reading', 'IC3_Repetition']
folder_list = ['data_healthy_v1', 'data_healthy_v2', 'data_ic3online_cognition', 'data_healthy_merged']
merged_data_folder ='/data_healthy_combined'
data_format = '.csv'

In [3]:
main_preprocessing(root_path, list_of_tasks, list_of_questionnaires, list_of_speech)

Starting preprocessing...
Cleaning normative data.Creating inclusion criteria...We removed 6 people who failed both Orientation and Pear Cancellation, from all tasks.
We removed 407 who indicated they have a neurological disorder, 91 who have a history of dementia and 92 who are younger than 40.
We removed 11 people who cheated.
We removed 17 people who failed both Orientation and Pear Cancellation, from all tasks.
We removed 33 who indicated they have a neurological disorder, and 4 who are younger than 40.
Done
Merging data across sites...Merged IC3_NVtrailMaking
Merged IC3_Orientation
Merged IC3_PearCancellation
Merged IC3_rs_digitSpan
Merged IC3_rs_spatialSpan
Merged IC3_rs_PAL
Merged IC3_rs_SRT
Merged IC3_rs_CRT
Merged IC3_SemanticJudgment
Merged IC3_i4i_IDED
Merged IC3_i4i_motorControl
Merged IC3_calculation
Merged IC3_GestureRecognition
Merged IC3_AuditorySustainedAttention
Merged IC3_BBCrs_blocks
Merged IC3_Comprehension
Merged IC3_rs_oddOneOut
Merged IC3_TaskRecall
Merged IC3_N

In [ ]:
list_of_tasks = list_of_tasks[:-2]


'/Users/dg519/Documents/normative_paper_github/data/data_healthy_combined'

In [5]:
task_name = 'IC3_NVtrailMaking'
inclusion_criteria = pd.read_csv('inclusion_criteria.csv')['user_id']

In [6]:
inclusion_criteria

0       6f181486-0e14-450e-bfe3-fb85e925adba
1       12e7f6c0-f77e-4064-88fc-abb9e1952693
2       54270eee-d9a7-4d44-b72f-f0137372acce
3       68732c3e-9d23-4212-ad3c-22acaddff7e0
4       4b21c478-5926-4c59-b37f-60d6a40ccf43
                        ...                 
6707       ic3study30079-sessionMT2-versionB
6708       ic3study30065-sessionMT3-versionB
6709         ic3study10013-session2-versionB
6710         ic3study30047-session1-versionB
6711         ic3study10017-session2-versionB
Name: user_id, Length: 6712, dtype: object

In [7]:
def remove_general_outliers(root_path, merged_data_folder, task_name, inclusion_criteria,  data_format, folder_structure=['/summary_data','/trial_data','/speech']):
    
    path_to_data = root_path + merged_data_folder
    print(f'{path_to_data}{folder_structure[0]}/{task_name}{data_format}')
    try:
        df = pd.read_csv(f'{path_to_data}/{folder_structure[0]}/{task_name}{data_format}', low_memory=False)
        df = df[df.user_id.isin(inclusion_criteria)]
        
        df_raw = pd.read_csv(f'{path_to_data}/{folder_structure[1]}/{task_name}_raw{data_format}', low_memory=False)
        df_raw = df_raw[df_raw.user_id.isin(inclusion_criteria)]
    except:
        print(f'Error in loading {task_name}. File might not exist.')
        return None,None

    df.drop_duplicates(subset=['user_id'],keep="last", inplace=True)
    df.drop(columns=['Unnamed: 0','Level','type','RespObject','sequenceObj', 'dynamicDifficulty'], inplace=True)
    df.reset_index(drop=True,inplace=True)

    if ('Unnamed: 0' in df_raw.columns):
        
        df_raw = df_raw.rename(columns={'Unnamed: 0':'Level_filter'})

    df_raw.drop_duplicates(subset=['user_id','Level_filter'],keep="last", inplace=True)
    df_raw.reset_index(drop=True,inplace=True)

    return df,df_raw

In [8]:
inclusion_criteria

0       6f181486-0e14-450e-bfe3-fb85e925adba
1       12e7f6c0-f77e-4064-88fc-abb9e1952693
2       54270eee-d9a7-4d44-b72f-f0137372acce
3       68732c3e-9d23-4212-ad3c-22acaddff7e0
4       4b21c478-5926-4c59-b37f-60d6a40ccf43
                        ...                 
6707       ic3study30079-sessionMT2-versionB
6708       ic3study30065-sessionMT3-versionB
6709         ic3study10013-session2-versionB
6710         ic3study30047-session1-versionB
6711         ic3study10017-session2-versionB
Name: user_id, Length: 6712, dtype: object

In [9]:
df,df_raw = remove_general_outliers(root_path, merged_data_folder, task_name, inclusion_criteria,  data_format, folder_structure)

/Users/dg519/Documents/normative_paper_github/data/data_healthy_combined/summary_data/IC3_NVtrailMaking.csv


In [10]:
df2,df_raw2 = remove_general_outliers(root_path, merged_data_folder, f'{task_name}2', inclusion_criteria, data_format, folder_structure)
df3,df_raw3 = remove_general_outliers(root_path, merged_data_folder, f'{task_name}3', inclusion_criteria, data_format, folder_structure)
df,df_raw = trailmaking_preproc(df,df2,df3,df_raw,df_raw2,df_raw3)

/Users/dg519/Documents/normative_paper_github/data/data_healthy_combined/summary_data/IC3_NVtrailMaking2.csv
/Users/dg519/Documents/normative_paper_github/data/data_healthy_combined/summary_data/IC3_NVtrailMaking3.csv


In [7]:
pwd

'/Users/dg519/Documents/normative_paper_github/data'

In [8]:
ls

data_healthy_cleaned/     data_ic3online_cognition/ data_patients_v2/
data_healthy_combined/    data_patients_cleaned/    output_data/
data_healthy_v1/          data_patients_merged/
data_healthy_v2/          data_patients_v1/


In [9]:
merged_data_folder

'/data_healthy_combined'

In [11]:
os.path.isdir(merged_data_folder[1:])

True

In [5]:
list_of_tasks = list_of_tasks[:-2]


In [4]:
ids_remote = general_outlier_detection_remoteSetting(root_path, remote_data_folders, folder_structure, screening_list = ['q_IC3_demographicsHealthy_questionnaire.csv', 'q_IC3_metacog_questionnaire.csv', 'IC3_Orientation.csv', 'IC3_PearCancellation.csv'])


We removed 6 people who failed both Orientation and Pear Cancellation, from all tasks.
We removed 407 who indicated they have a neurological disorder, 91 who have a history of dementia and 92 who are younger than 40.
We removed 11 people who cheated.


In [6]:
def general_outlier_detection_supervisedSetting(root_path, supervised_data_folders, folder_structure, screening_list = ['q_IC3_demographics_questionnaire.csv', 'IC3_Orientation.csv', 'IC3_PearCancellation.csv']): 

    # Read the data for v1

    os.chdir(root_path + supervised_data_folders[0] + folder_structure[0])

    df_dem_tp1 = pd.read_csv(screening_list[0], low_memory=False)
    df_orient_tp1 = pd.read_csv(screening_list[1], low_memory=False)
    df_pear_tp1 = pd.read_csv(screening_list[2], low_memory=False)

    # Read the data for v2

    os.chdir(root_path + supervised_data_folders[1] + folder_structure[0])

    df_dem_tp2 = pd.read_csv(screening_list[0], low_memory=False)
    df_orient_tp2 = pd.read_csv(screening_list[1], low_memory=False)
    df_pear_tp2 = pd.read_csv(screening_list[2], low_memory=False)

    # Concatenate the two timepoints

    df_dem = pd.concat([df_dem_tp1, df_dem_tp2], ignore_index=True)
    df_orient = pd.concat([df_orient_tp1, df_orient_tp2], ignore_index=True)
    df_pear = pd.concat([df_pear_tp1, df_pear_tp2], ignore_index=True)

    # Remove duplicates, NAs and reset index

    df_dem.drop_duplicates(subset=['user_id'],keep="first", inplace=True)
    df_dem.dropna(subset=['user_id'], inplace=True)
    df_dem.reset_index(drop=True, inplace=True)

    df_orient.drop_duplicates(subset=['user_id'],keep="first", inplace=True)
    df_orient.dropna(subset=['user_id'], inplace=True)
    df_orient.reset_index(drop=True, inplace=True)

    df_pear.drop_duplicates(subset=['user_id'],keep="first", inplace=True)
    df_pear.dropna(subset=['user_id'], inplace=True)
    df_pear.reset_index(drop=True, inplace=True)

    # Find the user_id who are in both df_pear and df_pear

    cleaned_ids_supervised = list(set(df_dem['user_id']).intersection(set(df_orient['user_id'])).intersection(set(df_pear['user_id'])))

    df_pear = df_pear[df_pear.user_id.isin(cleaned_ids_supervised)]  
    df_orient = df_orient[df_orient.user_id.isin(cleaned_ids_supervised)]  
    df_dem = df_dem[df_dem.user_id.isin(cleaned_ids_supervised)]  

    # Drop the user_id, if they fail the screening test

    ids_failed_screen = []
    for subs in cleaned_ids_supervised:
        if (df_orient[df_orient.user_id == subs].SummaryScore < 3).bool() and (df_pear[df_pear.user_id == subs].SummaryScore <= 0.80).bool():
            
            ids_failed_screen.append(subs)
            df_dem = df_dem.drop(df_dem[df_dem.user_id == subs].index).reset_index(drop=True)
            df_orient = df_orient.drop(df_orient[df_orient.user_id == subs].index).reset_index(drop=True)
            df_pear = df_pear.drop(df_pear[df_pear.user_id == subs].index).reset_index(drop=True)

    print(f'We removed {len(ids_failed_screen)} people who failed both Orientation and Pear Cancellation, from all tasks.')


    cleaned_ids_supervised = df_pear.user_id

    # Remove people who are not neurologically healthy

    to_remove = (df_dem.Q12_R != "SKIPPED") | (df_dem.Q14_R != "SKIPPED") | (df_dem.Q1_R < 40)    
    print(f'We removed {sum((df_dem.Q12_R != "SKIPPED") | (df_dem.Q14_R != "SKIPPED"))} who indicated they have a neurological disorder, and {sum(df_dem.Q1_R < 40)} who are younger than 40.')

    df_dem = df_dem[~to_remove].reset_index(drop=True)
    df_pear = df_pear[~to_remove].reset_index(drop=True)
    df_orient = df_orient[~to_remove].reset_index(drop=True)

    cleaned_ids_supervised = df_pear.user_id

    return cleaned_ids_supervised # Return participant ids that passed exclusion criteria


In [8]:
ids_supervised =  general_outlier_detection_supervisedSetting(root_path, supervised_data_folders, folder_structure, screening_list = ['q_IC3_demographics_questionnaire.csv', 'IC3_Orientation.csv', 'IC3_PearCancellation.csv'])


We removed 17 people who failed both Orientation and Pear Cancellation, from all tasks.
We removed 33 who indicated they have a neurological disorder, and 4 who are younger than 40.


In [ ]:
inclusion_criteria = pd.concat([ids_remote, ids_supervised], ignore_index=True)
inclusion_criteria.reset_index(drop=True,inplace=True)
if not os.path.isdir(merged_data_folder):
    os.mkdir(merged_data_folder)
inclusion_criteria.to_csv(f'{root_path}{merged_data_folder}/inclusion_criteria.csv', index=False)


In [ ]:
def merge_control_data_across_sites(root_path, folder_structure, supervised_data_folders, remote_data_folders, list_of_tasks,list_of_questionnaires,list_of_speech, data_format, merged_data_folder):
        
    os.chdir(root_path)

    # Create folder structure
    
    if os.path.isdir(merged_data_folder[1:]) == False:
        os.mkdir(merged_data_folder[1:])
    os.chdir(merged_data_folder[1:])

    if os.path.isdir(folder_structure[0][1:]) == False:
        os.mkdir(folder_structure[0][1:])
        
    if os.path.isdir(folder_structure[1][1:]) == False:
        os.mkdir(folder_structure[1][1:])
    
    if os.path.isdir(folder_structure[2][1:]) == False:
        os.mkdir(folder_structure[2][1:])
        
    
        

    # Merge data from clinical tests
    
    for taskName in list_of_tasks:
            
        
        summary_task_path = root_path + supervised_data_folders[0] + folder_structure[0]
        raw_task_path = root_path + supervised_data_folders[0] + folder_structure[1]
        
        df_v1 = pd.read_csv(f'{summary_task_path}/{taskName}{data_format}', low_memory=False)
        df_v1_raw = pd.read_csv((f'{raw_task_path}/{taskName}_raw{data_format}'), low_memory=False)
        
        summary_task_path = root_path + supervised_data_folders[1] + folder_structure[0]
        raw_task_path = root_path + supervised_data_folders[1] + folder_structure[1]
        
        # Special case for IDED task that has two versions
        
        if taskName == "IC3_i4i_IDED": 
            df_v2 = pd.read_csv((f'{summary_task_path}/{taskName}2{data_format}'), low_memory=False)
            df_v2_raw = pd.read_csv((f'{raw_task_path}/{taskName}2_raw{data_format}'), low_memory=False)
        else:   
            df_v2 = pd.read_csv(f'{summary_task_path}/{taskName}{data_format}', low_memory=False)
            df_v2_raw = pd.read_csv((f'{raw_task_path}/{taskName}_raw{data_format}'), low_memory=False)
        
        
        summary_task_path = root_path + remote_data_folders + folder_structure[0]
        raw_task_path = root_path + remote_data_folders + folder_structure[1]

        df_cog = pd.read_csv(f'{summary_task_path}/{taskName}{data_format}', low_memory=False)
        df_cog_raw = pd.read_csv((f'{raw_task_path}/{taskName}_raw{data_format}'), low_memory=False)
        
        df = pd.concat([df_v1, df_v2, df_cog], ignore_index=True)
        df_raw = pd.concat([df_v1_raw, df_v2_raw, df_cog_raw], ignore_index=True)
        
        output_folder = root_path + merged_data_folder
        
        df.to_csv(f'{output_folder}/{folder_structure[0]}/{taskName}{data_format}', index=False)
        df_raw.to_csv(f'{output_folder}/{folder_structure[1]}/{taskName}_raw{data_format}', index=False)
        print(f'Merged {taskName}')
        
    # Merge data from speech
        
    for taskName in list_of_speech:
        
        raw_speech_path = root_path + supervised_data_folders[0] + folder_structure[1]
        df_v1_raw = pd.read_csv((f'{raw_speech_path}/{taskName}_raw{data_format}'), low_memory=False)
        
        raw_speech_path = root_path + supervised_data_folders[1] + folder_structure[1]
        df_v2_raw = pd.read_csv((f'{raw_speech_path}/{taskName}_raw{data_format}'), low_memory=False)

        df_raw = pd.concat([df_v1_raw, df_v2_raw], ignore_index=True)
        
        output_folder = root_path + merged_data_folder
        df_raw.to_csv(f'{output_folder}/{folder_structure[1]}/{taskName}_raw.csv', index=False)
        print(f'Merged {taskName}')
        
    # Merge data from questionnaires

    for taskName in list_of_questionnaires:
        
        if taskName == 'q_IC3_demographics':
            
            summary_task_path = root_path + supervised_data_folders[0] + folder_structure[0]
            raw_task_path = root_path + supervised_data_folders[0] + folder_structure[1]
            
            df_v1 = pd.read_csv((f'{summary_task_path}/{taskName}Healthy_questionnaire.csv'), low_memory=False)
            df_v1_2 = pd.read_csv((f'{summary_task_path}/{taskName}_questionnaire.csv'), low_memory=False)
            df_v1_raw = pd.read_csv((f'{raw_task_path}/{taskName}_raw.csv'), low_memory=False)
            df_v1_raw_2 = pd.read_csv((f'{raw_task_path}/{taskName}Healthy_raw.csv'), low_memory=False)

            summary_task_path = root_path + supervised_data_folders[1] + folder_structure[0]
            raw_task_path = root_path + supervised_data_folders[1] + folder_structure[1]
 
            df_v2 = pd.read_csv((f'{summary_task_path}/{taskName}_questionnaire.csv'), low_memory=False)
            df_v2_raw = pd.read_csv((f'{raw_task_path}/{taskName}_raw.csv'), low_memory=False)
            
            summary_task_path = root_path + remote_data_folders + folder_structure[0]
            raw_task_path = root_path + remote_data_folders + folder_structure[1]

            df_cog = pd.read_csv((f'{summary_task_path}/{taskName}Healthy_questionnaire.csv'), low_memory=False)
            df_cog_raw = pd.read_csv((f'{raw_task_path}/{taskName}Healthy_raw.csv'), low_memory=False)

            df = pd.concat([df_v1, df_v1_2, df_v2, df_cog], ignore_index=True)
            df_raw = pd.concat([df_v1_raw, df_v1_raw_2, df_v2_raw, df_cog_raw], ignore_index=True)
            
        else:
            
            summary_task_path = root_path + supervised_data_folders[0] + folder_structure[0]
            raw_task_path = root_path + supervised_data_folders[0] + folder_structure[1]
            
            df_v1 = pd.read_csv(f'{summary_task_path}/{taskName}_questionnaire.csv', low_memory=False)
            df_v1_raw = pd.read_csv((f'{raw_task_path}/{taskName}_raw.csv'), low_memory=False)
            
            summary_task_path = root_path + supervised_data_folders[1] + folder_structure[0]
            raw_task_path = root_path + supervised_data_folders[1] + folder_structure[1]
            
            df_v2 = pd.read_csv(f'{summary_task_path}/{taskName}_questionnaire.csv', low_memory=False)
            df_v2_raw = pd.read_csv((f'{raw_task_path}/{taskName}_raw.csv'), low_memory=False)
            
            summary_task_path = root_path + remote_data_folders + folder_structure[0]
            raw_task_path = root_path + remote_data_folders + folder_structure[1]
            
            df_cog = pd.read_csv(f'{summary_task_path}/{taskName}_questionnaire.csv', low_memory=False)
            df_cog_raw = pd.read_csv((f'{raw_task_path}/{taskName}_raw.csv'), low_memory=False)
            
            df = pd.concat([df_v1, df_v2, df_cog], ignore_index=True)
            df_raw = pd.concat([df_v1_raw, df_v2_raw, df_cog_raw], ignore_index=True)
        
        
            
        output_folder = root_path + merged_data_folder

        df.to_csv(f'{output_folder}/{folder_structure[0]}/{taskName}{data_format}', index=False)
        df_raw.to_csv(f'{output_folder}/{folder_structure[1]}/{taskName}_raw.csv', index=False)
        print(f'Merged {taskName}')
    
    return None

In [43]:
merge_control_data_across_sites(root_path, folder_structure, supervised_data_folders, remote_data_folders, list_of_tasks,list_of_questionnaires,list_of_speech, data_format, merged_data_folder)


Merged IC3_Orientation
Merged IC3_PearCancellation
Merged IC3_rs_digitSpan
Merged IC3_rs_spatialSpan
Merged IC3_rs_PAL
Merged IC3_rs_SRT
Merged IC3_rs_CRT
Merged IC3_SemanticJudgment
Merged IC3_i4i_IDED
Merged IC3_i4i_motorControl
Merged IC3_calculation
Merged IC3_GestureRecognition
Merged IC3_AuditorySustainedAttention
Merged IC3_BBCrs_blocks
Merged IC3_Comprehension
Merged IC3_NVtrailMaking
Merged IC3_rs_oddOneOut
Merged IC3_TaskRecall
Merged IC3_NamingTest
Merged IC3_Reading
Merged IC3_Repetition
Merged q_IC3_demographics
Merged q_IC3_fatigue
Merged q_IC3_GDS
Merged q_IC3_apathy


In [45]:
def remove_general_outliers(root_path, merged_data_folder, task_name, inclusion_criteria,  data_format, folder_structure=['/summary_data','/trial_data','/speech']):
    
    path_to_data = root_path + merged_data_folder
    
    try:
        df = pd.read_csv(f'{path_to_data}/{folder_structure[0]}/{task_name}{data_format}', low_memory=False)
        df = df[df.user_id.isin(inclusion_criteria)]

        df_raw = pd.read_csv(f'{path_to_data}/{folder_structure[1]}/{task_name}_raw{data_format}', low_memory=False)
        df_raw = df_raw[df_raw.user_id.isin(inclusion_criteria)]
    except:
        print(f'Error in loading {task_name}. File might not exist.')
        return None,None

    df.drop_duplicates(subset=['user_id'],keep="last", inplace=True)
    df.drop(columns=['Unnamed: 0','Level','type','RespObject','sequenceObj', 'dynamicDifficulty'], inplace=True)
    df.reset_index(drop=True,inplace=True)

    if ('Unnamed: 0' in df_raw.columns):
        
        df_raw = df_raw.rename(columns={'Unnamed: 0':'Level_filter'})

    df_raw.drop_duplicates(subset=['user_id','Level_filter'],keep="last", inplace=True)
    df_raw.reset_index(drop=True,inplace=True)

    return df,df_raw


In [47]:
task_name = 'IC3_Orientation'

In [48]:
df,df_raw = remove_general_outliers(root_path, merged_data_folder, task_name, inclusion_criteria,  data_format, folder_structure)


In [50]:
def orientation_preproc(df,df_raw):

    df_raw.loc[df_raw.RT<400,'RT'] = np.nan
    df_raw.loc[df_raw.RT>10000,'RT'] = np.nan

    scores = [None] * len(df.user_id)
    errors = [None] * len(df.user_id)
    meanRTs =[None] * len(df.user_id)
    medianRTs =[None] * len(df.user_id)
    meanErrorRTs =[None] * len(df.user_id)
    meanCorrRTs =[None] * len(df.user_id)
    medianErrorRTs =[None] * len(df.user_id)
    medianCorrRTs =[None] * len(df.user_id)
    numNAs = [None] * len(df.user_id)

    for count,id in enumerate(df.user_id):
        df_raw_temp = df_raw[df_raw.user_id == id]
        
        date_raw = df_raw_temp.iloc[0].TimeRespEnabled
        datestart = datetime.datetime.fromtimestamp(float(date_raw)/1000).strftime('%Y-%m-%d %H:%M:%S')
        datestart = datestart.split(' ')[1].split(':')
        
        if any(x == int(datestart[0]) for x in [6, 12, 18, 0]) and (int(datestart[1]) < 30):
            df_raw_temp.loc[df_raw_temp.Level == 4, 'correct'] = True
        elif any(x == int(datestart[0]) for x in [5, 11, 17, 23]) and (int(datestart[1]) > 30):
            df_raw_temp.loc[df_raw_temp.Level == 4, 'correct'] = True
            
        errors[count] = (sum(df_raw_temp.correct == False))
        scores[count] = sum(df_raw_temp.correct)
        
        meanRTs[count] = np.nanmean(df_raw_temp.RT)
        meanErrorRTs[count] = np.nanmean(df_raw_temp[df_raw_temp.correct == False].RT) if (~df_raw_temp.correct).any() else np.nan
        meanCorrRTs[count] = np.nanmean(df_raw_temp[df_raw_temp.correct == True].RT) if (df_raw_temp.correct).any() else np.nan
        medianRTs[count] = np.nanmedian(df_raw_temp.RT)
        medianErrorRTs[count] = np.nanmedian(df_raw_temp[df_raw_temp.correct == False].RT) if (~df_raw_temp.correct).any() else np.nan
        medianCorrRTs[count] = np.nanmedian(df_raw_temp[df_raw_temp.correct == True].RT) if (df_raw_temp.correct).any() else np.nan
        numNAs[count] = df_raw_temp.RT.isna().sum() / len(df_raw_temp.RT)
        
    df_temp = pd.DataFrame({"user_id":df.user_id, "score":scores, "errors":errors, "meanRT":meanRTs, "medianRT":medianRTs, "medianErrorRT":medianErrorRTs, "medianCorrRT":medianCorrRTs, "meanErrorRT":meanErrorRTs, "meanCorrRT":meanCorrRTs, "numNAs":numNAs})

    df["SummaryScore"] = df_temp.score
    df["totalCorrect"] = df_temp.score
    df["errors"] = df_temp.errors
    df["medianRT"] = df_temp.medianRT
    df["meanRT"] = df_temp.meanRT
    df["medianCorrectRT"] = df_temp.medianCorrRT
    df["medianErrorRT"] = df_temp.medianErrorRT
    df["meanCorrectRT"] = df_temp.meanCorrRT
    df["meanErrorRT"] = df_temp.meanErrorRT

    exc = ((df.timeOffScreen > 10000) | (df.focusLossCount > 2) | (df_temp.numNAs > 0.6) | (df.SummaryScore < 2)) & (df.SummaryScore <4)
    df.drop(df[exc].index, inplace=True)
    df.reset_index(drop=True,inplace=True)
    df_raw = df_raw[df_raw.user_id.isin(df.user_id)]
    df_raw.reset_index(drop=True,inplace=True)

    return df,df_raw

In [51]:
df,df_raw = orientation_preproc(df,df_raw)


In [52]:
def output_preprocessed_data(df,df_raw, root_path, output_clean_folder, folder_structure, clean_file_extension, data_format):
    
    os.chdir(root_path)
    
    if not os.path.isdir(output_clean_folder[1:]):
        os.mkdir(output_clean_folder[1:])
        os.mkdir(f'.{output_clean_folder}{folder_structure[0]}')
        os.mkdir(f'.{output_clean_folder}{folder_structure[1]}')

    df.to_csv(f".{output_clean_folder}{folder_structure[0]}/{df.loc[0,'taskID']}{clean_file_extension}{data_format}", index=False)
    df_raw.to_csv(f".{output_clean_folder}{folder_structure[1]}/{df.loc[0,'taskID']}_raw{clean_file_extension}{data_format}")

    return None

In [54]:
clean_file_extension='_cleaned'

In [55]:
output_preprocessed_data(df,df_raw, root_path, output_clean_folder, folder_structure, clean_file_extension, data_format)

In [58]:
def demographics_preproc(root_path, merged_data_folder, questionnaire_name, inclusion_criteria, folder_structure, data_format, clean_file_extension):
    
    os.chdir(root_path + merged_data_folder)
    
    try:
        df_dem_summary = pd.read_csv(f'.{folder_structure[0]}/{questionnaire_name}{data_format}', low_memory=False)
        df_dem_summary = df_dem_summary[df_dem_summary.user_id.isin(inclusion_criteria)]
        
        df_dem = pd.read_csv(f'.{folder_structure[1]}/{questionnaire_name}_raw{data_format}', low_memory=False)
        df_dem = df_dem[df_dem.user_id.isin(inclusion_criteria)]    

    except:
        print(f'Error in loading {questionnaire_name}. File might not exist.')
        return None
    
    df_dem.drop(['Unnamed: 0'], axis=1, inplace=True)
    df_dem =df_dem.drop_duplicates(subset=['user_id','question'], keep='last').reset_index(drop=True)
    df_dem_summary =df_dem_summary.drop_duplicates(subset='user_id',keep='last').reset_index(drop=True)
    
    # Extract demographics of interest
    
    age = df_dem.groupby(['question']).get_group('<center>Howoldareyou?</center>').loc[:,['response','user_id']].reset_index(drop=True)
    gender = df_dem.groupby(['question']).get_group('<center>Whatisyourbiologicalsex?</center>').loc[:,['response','user_id']].reset_index(drop=True)
    education = df_dem.groupby(['question']).get_group('<center>Whatisyourhighestlevelofeducation?</center>').loc[:,['response','user_id']].reset_index(drop=True)
    device = df_dem.groupby(['question']).get_group('<center>Whatdeviceareyouusingatthemoment?</center>').loc[:,['response','user_id']].reset_index(drop=True)
    english = df_dem.groupby(['question']).get_group('<center>HowwouldyourateyourproficiencyinEnglish?</center>').loc[:,['response','user_id']].reset_index(drop=True)
    depression = df_dem.groupby(['question']).get_group('<center>Areyoucurrentlytakinganymedicationfordepression?</center>').loc[:,['response','user_id']].reset_index(drop=True)
    anxiety = df_dem.groupby(['question']).get_group('<center>Areyoucurrentlytakinganymedicationforanxiety?</center>').loc[:,['response','user_id']].reset_index(drop=True)
    dyslexia = df_dem.groupby(['question']).get_group('<center>DoyouhaveDyslexia,oranyotherproblemswithreadingandwriting?</center>').loc[:,['response','user_id']].reset_index(drop=True)
    risks = df_dem.groupby(['question']).get_group('<center>Haveyoueverbeentoldyouhavethefollowing?Tickallthatapplies</center>').loc[:,['response','user_id']].reset_index(drop=True)

    # Clean each variable
    
    age.response = pd.to_numeric(age.response)
    age.loc[age.response < 40,'response'] = np.nan
    
    gender.response = gender.response.replace(['53','55','65','78','71','72'],np.nan)
    gender.replace(['Male','Female'],[0,1], inplace=True)

    education.response = education.response.replace('1',np.nan)
    education.response = education.response.replace('Secondary/HighSchoolDiploma','Secondary/HighSchool-A-levels')
    education.response = education.response.replace('Primary/ElementarySchool','SecondarySchool-GCSE')
    education.replace(['SecondarySchool-GCSE','Secondary/HighSchool-A-levels','ProfessionalDegree','Bachelor\'sDegree','Master\'sDegree','PhD'],[0,1,1,2,3,3], inplace=True)
                
    device = device.merge(df_dem_summary.loc[:,['user_id','os']], on='user_id', how='outer')
    device.response = device.response.fillna(device.os)
    device.drop('os',axis=1,inplace=True)
    
    english.response.replace({'3': 1, '4': 1, 'No': np.nan, '2':1,'1':0}, inplace=True)
    depression.response.replace({'No':0, 'Yes':1, 'SKIPPED':0}, inplace=True)   
    anxiety.response.replace({'No':0, 'Yes':1, 'SKIPPED':0}, inplace=True)
    dyslexia.response.replace({'Yes':1,'No':0,'Tablet':0,'Touchscreen':0}, inplace=True)
    
    risks.drop_duplicates(keep='last', inplace=True)
    risks.replace(np.nan, ' ', inplace=True)
    risks.response = risks.response.str.lower()
    risks['diabetes'] = risks.response.apply(lambda x: 'diabetes' in x).replace([True,False], [1,0])
    risks['highbloodpressure'] = risks.response.apply(lambda x: 'highbloodpressure' in x).replace([True,False], [1,0])
    risks['highcholesterol'] = risks.response.apply(lambda x: ('highcholesterol' in x) or ('highcholesterole' in x)).replace([True,False], [1,0])
    risks['heartdisease'] = risks.response.apply(lambda x: 'heartdisease' in x).replace([True,False], [1,0])
    risks['kidneydisease'] = risks.response.apply(lambda x: 'kidneydisease' in x).replace([True,False], [1,0])
    risks['alcoholdependency'] = risks.response.apply(lambda x: 'alcoholdependency' in x).replace([True,False], [1,0])
    risks['over-weight'] = risks.response.apply(lambda x: ('over-weight' in x) or ('overweight' in x)).replace([True,False], [1,0])
    risks['long-termsmoker'] = risks.response.apply(lambda x: ('long-termsmoker' in x) or ('longtermsmoker' in x)).replace([True,False], [1,0])
    risks['ex-smoker'] = risks.response.apply(lambda x: ('ex-smoker' in x) or ('exsmoker' in x)).replace([True,False], [1,0])
    risks.loc[(risks['long-termsmoker'] & risks['ex-smoker']).astype(bool),'ex-smoker'] = 0
    risks.response = risks.iloc[:,2:].sum(axis=1)
    
    age.drop_duplicates(subset="user_id",keep='last',inplace=True)
    gender.drop_duplicates(subset="user_id", keep='last',inplace=True)
    education.drop_duplicates(subset="user_id", keep='last',inplace=True)
    device.drop_duplicates(subset="user_id", keep='last',inplace=True)
    english.drop_duplicates(subset="user_id", keep='last',inplace=True)
    depression.drop_duplicates(subset="user_id", keep='last',inplace=True)
    anxiety.drop_duplicates(subset="user_id", keep='last',inplace=True)
    dyslexia.drop_duplicates(subset="user_id", keep='last',inplace=True)
    risks.drop_duplicates(subset="user_id", keep='last',inplace=True)
    
    age.rename(columns={'response':'age'}, inplace=True)
    gender.rename(columns={'response':'gender'}, inplace=True)
    education.rename(columns={'response':'education'}, inplace=True)
    device.rename(columns={'response':'device'}, inplace=True)
    english.rename(columns={'response':'english'}, inplace=True)
    depression.rename(columns={'response':'depression'}, inplace=True)
    anxiety.rename(columns={'response':'anxiety'}, inplace=True)
    dyslexia.rename(columns={'response':'dyslexia'}, inplace=True)
    risks.rename(columns={'response':'risks'}, inplace=True)
    
    age.dropna(inplace=True)
    gender.dropna(inplace=True)
    education.dropna(inplace=True)
    device.dropna(inplace=True)
    english.dropna(inplace=True)
    depression.dropna(inplace=True)
    anxiety.dropna(inplace=True)
    dyslexia.dropna(inplace=True)
    risks.dropna(inplace=True)
              
    # Merge and format
    
    healthy_demographics = age.merge(gender,on='user_id').merge(education,on='user_id').merge(device,on='user_id').merge(english,on='user_id').merge(depression,on='user_id').merge(anxiety,on='user_id').merge(risks,on='user_id').merge(dyslexia,on='user_id')
    healthy_demographics.education = healthy_demographics.education.astype(int)
    
    one_hot_encoded_data = pd.get_dummies(healthy_demographics, columns = ['device', 'education'])
    one_hot_encoded_data.rename(columns={'education_1':'education_Alevels', 'education_2':'education_bachelors','education_3':'education_postBachelors'}, inplace=True)
    one_hot_encoded_data.rename(columns={'device_1':'device_tablet', 'device_0':'device_phone'}, inplace=True)
    one_hot_encoded_data.rename(columns={'english':'english_secondLanguage'}, inplace=True)
    one_hot_encoded_data.replace({True:1, False:0}, inplace=True)
    one_hot_encoded_data.loc[:,'gender':'education_postBachelors'] = one_hot_encoded_data.loc[:,'gender':'education_postBachelors'] -0.5

    # Save 
    
    one_hot_encoded_data.to_csv(f'.{folder_structure[0]}/{questionnaire_name}{clean_file_extension}{data_format}', index=False)
    
    return one_hot_encoded_data


In [57]:
questionnaire_name = 'q_IC3_demographics'

In [60]:
df_demographics = demographics_preproc(root_path, merged_data_folder, questionnaire_name, inclusion_criteria, folder_structure, data_format,clean_file_extension)


In [1]:
from preproc_functions_patients import *

In [2]:
root_path = '/Users/dg519/Documents/normative_paper_github/data'
list_of_tasks = ['IC3_Orientation', 'IC3_PearCancellation', 'IC3_rs_digitSpan', 'IC3_rs_spatialSpan', 'IC3_rs_PAL', 'IC3_rs_SRT', 'IC3_rs_CRT', 'IC3_SemanticJudgment', 'IC3_i4i_IDED', 'IC3_i4i_motorControl','IC3_calculation', 'IC3_GestureRecognition', 'IC3_AuditorySustainedAttention','IC3_BBCrs_blocks', 'IC3_Comprehension', 'IC3_NVtrailMaking','IC3_rs_oddOneOut', 'IC3_TaskRecall']
list_of_questionnaires = ['q_IC3_demographics', 'q_IC3_fatigue','q_IC3_GDS','q_IC3_apathy', 'q_IC3_IADL']
list_of_speech = ['IC3_NamingTest','IC3_Reading', 'IC3_Repetition']
patient_data_folders=['/data_patients_v1','/data_patients_v2']
folder_structure=['/summary_data','/trial_data','/speech']
output_clean_folder ='/data_patients_cleaned'
merged_data_folder ='/data_patients_merged'
clean_file_extension='_cleaned'
data_format='.csv'
clinical_information = '/Users/dg519/Documents/normative_paper_github/data/output_data/patients_database260924.xlsx'

In [3]:
df= main_preprocessing(root_path, list_of_tasks, list_of_questionnaires, list_of_speech, clinical_information)

Starting preprocessing...
Merging data across sites...Merged IC3_Orientation
Merged IC3_PearCancellation
Merged IC3_rs_digitSpan
Merged IC3_rs_spatialSpan
Merged IC3_rs_PAL
Merged IC3_rs_SRT
Merged IC3_rs_CRT
Merged IC3_SemanticJudgment
Merged IC3_i4i_IDED
Merged IC3_i4i_motorControl
Merged IC3_calculation
Merged IC3_GestureRecognition
Merged IC3_AuditorySustainedAttention
Merged IC3_BBCrs_blocks
Merged IC3_Comprehension
Merged IC3_NVtrailMaking
Merged IC3_rs_oddOneOut
Merged IC3_TaskRecall
Merged IC3_NamingTest
Merged IC3_Reading
Merged IC3_Repetition
Merged q_IC3_demographics
Merged q_IC3_fatigue
Merged q_IC3_GDS
Merged q_IC3_apathy
Merged q_IC3_IADL
Done
Pre-processing IC3_Orientation...Done
Pre-processing IC3_PearCancellation...Done
Pre-processing IC3_rs_digitSpan...Done
Pre-processing IC3_rs_spatialSpan...Done
Pre-processing IC3_rs_PAL...Done
Pre-processing IC3_rs_SRT...Done
Pre-processing IC3_rs_CRT...Done
Pre-processing IC3_SemanticJudgment...Done
Pre-processing IC3_i4i_IDED...D